In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [5]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'rated_capacity_amperage'

In [6]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [10]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='RatedCapacity'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

919


In [11]:
# df[field].explode().value_counts()
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, RatedCapacity]
Index: []

In [12]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [13]:
# df['MaxPressure'].explode().value_counts()

In [26]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(\d+)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'1\/16"?|1\/4"?|1\/8"?|3\/16"?|3\/32"?|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

# pat=r'(?i)(m)|()'
# df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')]
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm',' mm',str(x))).apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r'\.0','',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col]
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
matchcustom[curation_col].explode().value_counts()

789


Series([], Name: Q:wall_thickness_sae, dtype: int64)

In [34]:
not_match=df[(df[field].astype(str)=='nan')]
# print(len(not_match[(not_match['ld'].astype(str)=='[]')|(not_match['name'].astype(str)!='[]')]))
two=not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]
# print(len(two))
# two[curation_col]=two['name'].apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))
# matchtwo=two[['external_id',curation_col]]
# print(len(matchtwo))
two

buckets external_id  \
0     Compressors Accessories & Replacement Parts       40511   
1     Compressors Accessories & Replacement Parts       40490   
2     Compressors Accessories & Replacement Parts       40492   
3     Compressors Accessories & Replacement Parts       40491   
4     Compressors Accessories & Replacement Parts       40493   
...                                           ...         ...   
4462  Compressors Accessories & Replacement Parts    405784RE   
4463  Compressors Accessories & Replacement Parts    405786RE   
4464  Compressors Accessories & Replacement Parts    405787RE   
4465  Compressors Accessories & Replacement Parts     40578RE   
4466  Compressors Accessories & Replacement Parts     509005U   

                                                                         product_name  \
0                                                        1/16 Viton O-Ring Assortment   
1                                              1/16 - 1/8 Buna-N 90 O-Ring Assortment   
2                                                    3/32 Buna-N 90 O-Ring Assortment   
3                                                    1/16 Buna-N 90 O-Ring Assortment   
4                                                     1/8 Buna-N 90 O-Ring Assortment   
...                                                                               ...   
4462       Refill Kit for 1/4 - 5/8 Kim-Krimp 'Z' Series Face Seal Fitting Assortment   
4463                 Refill Kit for 1/4 - 1/2 Kim-Krimp 'Z' Series Fitting Assortment   
4464          Refill Kit for 1/4 - 1/2 Kim-Krimp 'Z' Series O-Ring Fitting Assortment   
4465  Refill Kit for Straight Thread O-Ring Kim-Krimp Hose End Assortment (1/4 - 1/2)   
4466                             Buna-N 90 O-Ring Assortment with O-Ring Gauge Bundle   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            75 Durometer. Manufactured from fluorocarbon rubber. Temperature range: -20° to 400°F. Has maximum resistance to deterioration by heat and fluids. Recognized by brown color.   
1                                                                                                                                                                                   

In [28]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [29]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [30]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [35]:
na=df[(df['name'].astype(str)=='[]')&(df[field].astype(str)=='nan')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))
na

2574


buckets external_id  \
9     Compressors Accessories & Replacement Parts       40497   
11    Compressors Accessories & Replacement Parts      404971   
12    Compressors Accessories & Replacement Parts     40497EZ   
13    Compressors Accessories & Replacement Parts       40498   
17    Compressors Accessories & Replacement Parts      405061   
...                                           ...         ...   
4444  Compressors Accessories & Replacement Parts   MC3309X18   
4445  Compressors Accessories & Replacement Parts   MC3309X14   
4446  Compressors Accessories & Replacement Parts   MC3309X20   
4452  Compressors Accessories & Replacement Parts      521009   
4453  Compressors Accessories & Replacement Parts    405752RE   

                                              product_name  \
9            Blue&Green Air Conditioning O-Ring Assortment   
11                Green Air Conditioning O-Ring Assortment   
12           Blue&Green Air Conditioning O-Ring Assortment   
13    Blue, Black&Green Air Conditioning O-Ring Assortment   
17    Caterpillar O-Ring, Face-Seal&O-Ring Boss Assortment   
...                                                    ...   
4444                                 M18 x 1.5 DIN Coupler   
4445                                 M14 x 1.5 DIN Coupler   
4446                                 M20 x 1.5 DIN Coupler   
4452                            Inline Air Tool Lubricator   
4453  Refill Kit for Synflex 903 Series Fitting Assortment   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
9                                                                                                                                                                                                                                                                                                                                                               Three different O-Ring colors, in different material construction, are available. Blue (neoprene) & Green (HSN) O-Rings for use with new R-134A refrigerant can also be used in applications with older R-12 refrigerant. Black O-Rings for use with R-12 refrigerant cannot be used with R-134A refrigerant.   
11                                                                                                                                                                                                                                                                                                                                                              Three different O-Ring colors, in different material construction, are available. Blue (neoprene) & Green (HSN) O-Rings for use with new R-134A refrigerant can also be used in applications with older R-12 refrigerant. Black O-Rings for use with R-12 refrigerant cannot be used with R-134A refrigerant.   
12                                                                                                                                                                                                                                                                                                                                                              Three different O-Ring colors, in different material construction, are available. Blue (neoprene) & Green (HSN) O-Rings for use with new R-134A refrigerant can also be used in applications with

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [36]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [37]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [30]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 